In [1]:
import pyspark
import findspark
findspark.find()
from pyspark.sql import SparkSession
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import Normalizer, VectorAssembler
from pyspark.ml.pipeline import Pipeline

In [2]:
import os
os.getcwd()

'C:\\Users\\bishw\\OneDrive\\Spark_for_Machine_Learning\\Logistic_Regression'

In [3]:
spark=SparkSession.builder.getOrCreate()

In [4]:
df=spark.read.csv('customer_churn.csv', inferSchema=True, header=True)

In [5]:
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [6]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [7]:
df=df.select(['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites','Churn'])

In [8]:
df.show()

+----+--------------+---------------+-----+---------+-----+
| Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|
+----+--------------+---------------+-----+---------+-----+
|42.0|       11066.8|              0| 7.22|      8.0|    1|
|41.0|      11916.22|              0|  6.5|     11.0|    1|
|38.0|      12884.75|              0| 6.67|     12.0|    1|
|42.0|       8010.76|              0| 6.71|     10.0|    1|
|37.0|       9191.58|              0| 5.56|      9.0|    1|
|48.0|      10356.02|              0| 5.12|      8.0|    1|
|44.0|      11331.58|              1| 5.23|     11.0|    1|
|32.0|       9885.12|              1| 6.92|      9.0|    1|
|43.0|       14062.6|              1| 5.46|     11.0|    1|
|40.0|       8066.94|              1| 7.11|     11.0|    1|
|30.0|      11575.37|              1| 5.22|      8.0|    1|
|45.0|       8771.02|              1| 6.64|     11.0|    1|
|45.0|       8988.67|              1| 4.84|     11.0|    1|
|40.0|       8283.32|              1|  5

In [9]:
vec_df=VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'], outputCol='features',handleInvalid='skip')

In [10]:
norm=Normalizer(inputCol='features',outputCol='norm_features')

In [11]:
mlp=MultilayerPerceptronClassifier(featuresCol='norm_features', labelCol='Churn', layers=[5,10,10,2])

In [12]:
evalu=MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='Churn')

In [13]:
pipe=Pipeline(stages=[vec_df, norm, mlp])

In [15]:
train, test= df.randomSplit([0.7,0.3])

In [16]:
dl=pipe.fit(train)

In [17]:
pred=dl.transform(test)

In [18]:
evalu.evaluate(pred)

0.7748418092336536

In [19]:
pred.show()

+----+--------------+---------------+-----+---------+-----+--------------------+--------------------+--------------------+--------------------+----------+
| Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|            features|       norm_features|       rawPrediction|         probability|prediction|
+----+--------------+---------------+-----+---------+-----+--------------------+--------------------+--------------------+--------------------+----------+
|22.0|      11254.38|              1| 4.96|      8.0|    0|[22.0,11254.38,1....|[0.00195479006264...|[-0.2682693024667...|[0.80535416433654...|       0.0|
|26.0|       8939.61|              0| 4.54|      7.0|    0|[26.0,8939.61,0.0...|[0.00290839071419...|[-0.2185906905889...|[0.82045140281650...|       0.0|
|27.0|        8628.8|              1|  5.3|      7.0|    0|[27.0,8628.8,1.0,...|[0.00312903922491...|[-0.2142246657368...|[0.82175861216236...|       0.0|
|29.0|       5900.78|              1| 5.56|      8.0|    0|[29.0,5900.